**Train on whole dataset and test on test set**

In [1]:
import os
import pandas as pd 
import numpy as np

from expert_tree import get_expert_tree_results, Expert_Tree
#from wrapper import best_first_search_mg

from sklearn.metrics import confusion_matrix

In [2]:
test_set_dir = 'r21_PRE_data_for_ET_validation.csv'
data_test = pd.read_csv(test_set_dir)
#data_test.head(5)
print(data_test.shape)

(134, 12)


In [3]:
data_test

,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,LVC,ChestWallSwelling
0,B001,0,0,0,0,0,0,0,0,0,-0.011774,0
1,B002,1,0,0,0,0,0,0,1,0,0.020581,0
2,B003,0,0,0,0,0,0,0,0,0,0.074392,0
3,B004,0,0,0,0,0,0,0,0,0,0.045896,0
4,B005,0,0,0,0,0,0,2,2,0,-0.035366,0
5,B006,0,0,0,0,0,0,0,0,0,0.065385,0
6,B007,0,0,0,0,1,0,0,1,0,0.027951,0
7,B008,0,0,0,0,0,0,0,0,0,-0.031071,0
8,B009,0,0,0,0,0,0,0,0,0,0.045231,0
9,B010,0,0,0,0,0,0,0,0,0,-0.066056,0


In [4]:

# fill 0 for missing data
for column in ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'ChestWallSwelling']:
    data_test[column].fillna(0, inplace=True)
# drop data missing important feature
data_test = data_test.dropna(subset=['TIME_LAPSE', 'LVC'])
# delete Username
data_test = data_test.drop(columns='Username')
# convert string to float
#data_test = data_test.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data_test = data_test.astype(np.float64)
# drop rows containing nan
data_test = data_test.dropna()
# drop data within a 6 months
#data_test = data_test[data_test.TIME_LAPSE >=0.5]
# add log of time elapsed
#data_test['TIME_LAPSE_LOG'] = np.log(data_test['TIME_LAPSE'])

print('shape:')
print(data_test.shape)

shape:
(134, 11)


In [5]:
#put on dummy values for fluid
data_test['fluid_total'] = [None]*data_test.shape[0]

In [6]:
data_test

,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,LVC,ChestWallSwelling,fluid_total
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011774,0.0,None
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.020581,0.0,None
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074392,0.0,None
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045896,0.0,None
4,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,-0.035366,0.0,None
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065385,0.0,None
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.027951,0.0,None
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.031071,0.0,None
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045231,0.0,None
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.066056,0.0,None


In [7]:
# generate 3 labels
expert_result, expert_labels, _ = get_expert_tree_results(data_test, False,3)
# Y_TEST is expert tree results

In [8]:
print(expert_result)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
truth_labels = [0]*data_test.shape[0]

In [ ]:
#They are all correct

EVALUATE THE RESULTS

In [10]:
from sklearn.metrics import confusion_matrix


In [11]:
t = confusion_matrix(truth_labels, expert_result, labels=[0,1,2]).ravel()
t = confusion_matrix(truth_labels, expert_result, labels=[0,1,2])
'''
print(tn, fp, fn, tp)
validation_accuracy = (tn+tp)/(tn+tp+fn+fp)
sensitivity = tp/(fn+tp)
specificty= tn/(tn+fp)
print(validation_accuracy, sensitivity,specificty )
'''
t

array([[133,   1,   0],
       [  0,   0,   0],
       [  0,   0,   0]], dtype=int64)

In [13]:
#def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
#def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
#def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
#def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]

In [12]:
data_test['clinical_label'] = truth_labels
data_test['expert_tree_label'] = expert_labels
data_test['expert_tree_result'] = expert_result

In [15]:
#data_test2['expert_tree_old'] = expert_tree_result2

In [13]:
result_df=data_test[data_test['clinical_label']!=data_test['expert_tree_result']]

In [14]:
result_df

,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,LVC,ChestWallSwelling,fluid_total,clinical_label,expert_tree_label,expert_tree_result
94,0.0,0.0,4.0,3.0,0.0,2.0,2.0,11.0,0.0,0.004539,4.0,None,0,Mild,1


In [15]:
new_column_names ={"ArmSwelling":"AS", "BreastSwelling":"BS", "DISCOMFORT":"Neuropathic", "ChestWallSwelling":"CS"}

In [16]:
result_df.rename(columns=new_column_names)

,Mobility,AS,BS,Skin,PAS,FHT,Neuropathic,SYM_COUNT,TIME_LAPSE,LVC,CS,fluid_total,clinical_label,expert_tree_label,expert_tree_result
94,0.0,0.0,4.0,3.0,0.0,2.0,2.0,11.0,0.0,0.004539,4.0,None,0,Mild,1
